In [1]:
# Cell 1: Import required libraries
import re
import random
import json
from datetime import datetime

print("✅ Libraries imported successfully!")
print(f"📅 Today's date: {datetime.now().strftime('%Y-%m-%d')}")

✅ Libraries imported successfully!
📅 Today's date: 2026-02-22


In [2]:
# Cell 2: Create our knowledge base
responses = {
    "greeting": {
        "patterns": ["hello", "hi", "hey", "good morning", "good afternoon", "good evening"],
        "responses": [
            "Hello! How can I help you today?",
            "Hi there! What can I do for you?",
            "Hey! Great to see you!",
            "Hello! Welcome to our customer support!"
        ]
    },
    "hours": {
        "patterns": ["hours", "open", "close", "timing", "when are you", "working hours", "business hours"],
        "responses": [
            "Our customer service hours are Monday-Friday, 9 AM to 6 PM EST.",
            "We're open weekdays from 9 AM to 6 PM Eastern Time.",
            "You can reach us Mon-Fri, 9am-6pm EST. We're closed on weekends."
        ]
    },
    "return": {
        "patterns": ["return", "refund", "exchange", "send back", "money back", "return policy"],
        "responses": [
            "You can return items within 30 days of purchase with original receipt.",
            "Our return policy: 30-day full refund, 31-60 days store credit only.",
            "To start a return, please email us your order number and reason."
        ]
    },
    "shipping": {
        "patterns": ["shipping", "delivery", "ship", "deliver", "track order", "where is my order"],
        "responses": [
            "Standard shipping takes 3-5 business days. Express shipping is 1-2 days.",
            "Free shipping on orders over $50!",
            "You can track your order using the tracking number in your email."
        ]
    },
    "payment": {
        "patterns": ["pay", "payment", "credit card", "debit card", "paypal", "installment"],
        "responses": [
            "We accept Visa, Mastercard, American Express, and PayPal.",
            "You can pay using credit/debit cards or PayPal.",
            "We also offer installment payments through Klarna."
        ]
    },
    "contact": {
        "patterns": ["contact", "email", "phone", "call", "reach", "customer service", "support"],
        "responses": [
            "You can email us at support@mycompany.com or call 1-800-123-4567",
            "Our support email: support@mycompany.com | Phone: 1-800-123-4567",
            "Live chat available on our website 24/7, or email us anytime!"
        ]
    },
    "product": {
        "patterns": ["product", "item", "what do you sell", "products", "catalog", "merchandise"],
        "responses": [
            "We sell electronics, clothing, home goods, and accessories.",
            "Check out our website for our full product catalog!",
            "We have over 10,000 products across 20 categories."
        ]
    },
    "price": {
        "patterns": ["price", "cost", "how much", "pricing", "expensive", "cheap", "discount"],
        "responses": [
            "Prices vary by product. Check our website for specific pricing.",
            "We have items ranging from $10 to $1000. What's your budget?",
            "We're currently running a 20% off sale on select items!"
        ]
    },
    "goodbye": {
        "patterns": ["bye", "goodbye", "see you", "talk to you later", "thanks bye", "have a good day"],
        "responses": [
            "Goodbye! Have a great day!",
            "Thanks for chatting! Come back anytime!",
            "Bye! Feel free to reach out if you need anything else!"
        ]
    },
    "thanks": {
        "patterns": ["thank", "thanks", "appreciate", "grateful"],
        "responses": [
            "You're welcome! Happy to help!",
            "My pleasure! Anything else I can assist with?",
            "Glad I could help!"
        ]
    },
    "default": {
        "responses": [
            "I'm not sure I understand. Could you rephrase that?",
            "Interesting question! Let me connect you with a human agent.",
            "I'm still learning. Can you ask in a different way?",
            "I don't have an answer for that yet. Would you like to speak with a person?"
        ]
    }
}

print(f"✅ Created {len(responses)} intent categories!")

✅ Created 11 intent categories!


In [3]:
# Cell 3: Simple pattern matching function
def match_intent_simple(user_input):
    """Simple rule-based intent matching"""
    user_input = user_input.lower()
    
    # Check each intent
    for intent, data in responses.items():
        if intent == 'default':
            continue
            
        # Check if any pattern matches
        for pattern in data.get('patterns', []):
            if pattern in user_input:
                return intent
    
    return 'default'

# Test it
test_inputs = ["hello", "what are your hours", "I want to return something", "thank you"]
for test in test_inputs:
    intent = match_intent_simple(test)
    print(f"'{test}' → Intent: {intent}")

'hello' → Intent: greeting
'what are your hours' → Intent: hours
'I want to return something' → Intent: greeting
'thank you' → Intent: thanks


In [4]:
# Cell 4: Better pattern matching using regex
def match_intent_regex(user_input):
    """Improved intent matching using word boundaries"""
    user_input = user_input.lower()
    
    for intent, data in responses.items():
        if intent == 'default':
            continue
            
        for pattern in data.get('patterns', []):
            # Use regex to match whole words only
            if re.search(r'\b' + re.escape(pattern) + r'\b', user_input):
                return intent
    
    return 'default'

# Test with word boundaries
test_phrases = ["shipping time", "I love shipping", "return policy", "I will return"]
for test in test_phrases:
    intent = match_intent_regex(test)
    responses_list = responses[intent]['responses'] if intent != 'default' else responses['default']['responses']
    print(f"'{test}' → Intent: {intent}")

'shipping time' → Intent: shipping
'I love shipping' → Intent: shipping
'return policy' → Intent: return
'I will return' → Intent: return


In [5]:
# Cell 5: Add conversation memory
class ChatbotMemory:
    def __init__(self):
        self.conversation_history = []
        self.last_intent = None
        self.user_name = None
        self.order_number = None
        
    def add_message(self, role, message):
        """Add a message to history"""
        self.conversation_history.append({
            'role': role,
            'message': message,
            'timestamp': datetime.now().strftime('%H:%M:%S')
        })
    
    def get_history(self, last_n=5):
        """Get last n messages"""
        return self.conversation_history[-last_n:]
    
    def set_context(self, key, value):
        """Store context information"""
        setattr(self, key, value)
    
    def get_context(self, key):
        """Retrieve context information"""
        return getattr(self, key, None)

# Initialize memory
memory = ChatbotMemory()
print("✅ Chatbot memory initialized!")

✅ Chatbot memory initialized!


In [6]:
# Cell 6: Complete chatbot class
class CustomerServiceBot:
    def __init__(self, responses_data, memory):
        self.responses = responses_data
        self.memory = memory
        self.conversation_count = 0
        
    def get_response(self, user_input):
        """Main method to get bot response"""
        
        # Add to conversation count
        self.conversation_count += 1
        
        # Store in memory
        self.memory.add_message('user', user_input)
        
        # Get intent
        intent = match_intent_regex(user_input)
        self.memory.last_intent = intent
        
        # Check for name extraction (bonus feature)
        if 'my name is' in user_input.lower():
            name = user_input.lower().split('my name is')[-1].strip()
            self.memory.user_name = name
            return f"Nice to meet you, {name}! How can I help you today?"
        
        # Check for order number
        if 'order' in user_input.lower() and '#' in user_input:
            # Simple order number extraction
            words = user_input.split()
            for word in words:
                if '#' in word:
                    self.memory.order_number = word.replace('#', '')
                    return f"I've noted your order #{self.memory.order_number}. How can I assist with it?"
        
        # Get response for intent
        if intent != 'default':
            response = random.choice(self.responses[intent]['responses'])
        else:
            response = random.choice(self.responses['default']['responses'])
        
        # Personalize if we know the name
        if self.memory.user_name and random.random() > 0.7:  # 30% chance
            response = f"{self.memory.user_name}, {response.lower()}"
        
        # Store bot response
        self.memory.add_message('bot', response)
        
        return response
    
    def show_stats(self):
        """Show conversation statistics"""
        print(f"\n📊 Conversation Stats:")
        print(f"Total messages: {len(self.memory.conversation_history)}")
        print(f"User name: {self.memory.user_name or 'Not provided'}")
        print(f"Last intent: {self.memory.last_intent}")
        print(f"Order number: {self.memory.order_number or 'None'}")
        
# Create bot instance
bot = CustomerServiceBot(responses, memory)
print("✅ Customer Service Bot created successfully!")

✅ Customer Service Bot created successfully!


In [7]:
# Cell 7: Test the bot with sample conversations
test_conversations = [
    "hello",
    "my name is John",
    "what are your hours",
    "I need to return something",
    "thank you",
    "goodbye"
]

print("🤖 Testing Chatbot Responses:\n")
for user_input in test_conversations:
    print(f"You: {user_input}")
    response = bot.get_response(user_input)
    print(f"Bot: {response}\n")
    print("-" * 50)

# Show stats
bot.show_stats()

🤖 Testing Chatbot Responses:

You: hello
Bot: Hi there! What can I do for you?

--------------------------------------------------
You: my name is John
Bot: Nice to meet you, john! How can I help you today?

--------------------------------------------------
You: what are your hours
Bot: We're open weekdays from 9 AM to 6 PM Eastern Time.

--------------------------------------------------
You: I need to return something
Bot: Our return policy: 30-day full refund, 31-60 days store credit only.

--------------------------------------------------
You: thank you
Bot: My pleasure! Anything else I can assist with?

--------------------------------------------------
You: goodbye
Bot: Goodbye! Have a great day!

--------------------------------------------------

📊 Conversation Stats:
Total messages: 11
User name: john
Last intent: goodbye
Order number: None


In [ ]:
# Cell 8: Live chat with your bot!
print("=" * 60)
print("🤖 CUSTOMER SERVICE CHATBOT - LIVE MODE")
print("=" * 60)
print("Type 'quit' to exit, 'stats' to see conversation stats")
print("-" * 60)

while True:
    user_input = input("\nYou: ")
    
    if user_input.lower() == 'quit':
        print("\nBot: Thanks for chatting! Have a great day! 👋")
        break
    elif user_input.lower() == 'stats':
        bot.show_stats()
        continue
    elif user_input.lower() == 'history':
        print("\n📝 Recent conversation:")
        for msg in memory.get_history():
            print(f"[{msg['timestamp']}] {msg['role'].upper()}: {msg['message']}")
        continue
    
    response = bot.get_response(user_input)
    print(f"Bot: {response}")

🤖 CUSTOMER SERVICE CHATBOT - LIVE MODE
Type 'quit' to exit, 'stats' to see conversation stats
------------------------------------------------------------
Bot: I don't have an answer for that yet. Would you like to speak with a person?
Bot: Hello! Welcome to our customer support!
Bot: I'm still learning. Can you ask in a different way?
Bot: Our customer service hours are Monday-Friday, 9 AM to 6 PM EST.
Bot: I'm still learning. Can you ask in a different way?
Bot: Interesting question! Let me connect you with a human agent.
Bot: john, i don't have an answer for that yet. would you like to speak with a person?


In [ ]:
# Cell 9: Install NLTK data (run once)
import nltk
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

# Download required NLTK data
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag

print("✅ NLP libraries loaded!")

In [ ]:
# Cell 10: NLP text processor
class NLPProcessor:
    def __init__(self):
        self.lemmatizer = WordNetLemmatizer()
        self.stop_words = set(stopwords.words('english'))
        
    def preprocess(self, text):
        """Clean and prepare text for analysis"""
        # Tokenize
        tokens = word_tokenize(text.lower())
        
        # Remove stopwords and punctuation
        tokens = [token for token in tokens 
                 if token.isalnum() and token not in self.stop_words]
        
        # Lemmatize
        tokens = [self.lemmatizer.lemmatize(token) for token in tokens]
        
        return tokens
    
    def extract_entities(self, text):
        """Simple entity extraction"""
        tokens = word_tokenize(text)
        pos_tags = pos_tag(tokens)
        
        entities = {
            'names': [],
            'numbers': [],
            'email': None,
            'phone': None
        }
        
        for word, tag in pos_tags:
            # Extract names (proper nouns)
            if tag == 'NNP':
                entities['names'].append(word)
            
            # Extract numbers
            if word.isdigit():
                entities['numbers'].append(word)
            
            # Simple email pattern
            if '@' in word and '.' in word:
                entities['email'] = word
            
            # Simple phone pattern (10 digits)
            if word.isdigit() and len(word) == 10:
                entities['phone'] = word
        
        return entities

# Initialize NLP processor
nlp = NLPProcessor()
print("✅ NLP Processor ready!")

In [ ]:
# Cell 11: Test NLP processing
test_texts = [
    "My name is John Smith and I live in New York",
    "My email is john@gmail.com and phone is 1234567890",
    "I want to return order #12345",
    "The products are running and they ran quickly"
]

print("🔬 NLP Processing Test:\n")
for text in test_texts:
    print(f"Original: {text}")
    print(f"Processed: {nlp.preprocess(text)}")
    print(f"Entities: {nlp.extract_entities(text)}")
    print("-" * 50)

In [ ]:
# Cell 12: Save conversation to file
import json
from datetime import datetime

def save_conversation(memory, filename=None):
    """Save conversation to JSON file"""
    if filename is None:
        filename = f"conversation_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
    
    data = {
        'timestamp': datetime.now().isoformat(),
        'conversation': memory.conversation_history,
        'user_name': memory.user_name,
        'order_number': memory.order_number,
        'total_messages': len(memory.conversation_history)
    }
    
    with open(filename, 'w') as f:
        json.dump(data, f, indent=2)
    
    print(f"✅ Conversation saved to {filename}")
    return filename

# Save current conversation
save_conversation(memory)

In [ ]:
# Cell 13: Load a saved conversation
def load_conversation(filename):
    """Load conversation from JSON file"""
    try:
        with open(filename, 'r') as f:
            data = json.load(f)
        print(f"✅ Loaded conversation from {filename}")
        print(f"📅 Date: {data['timestamp']}")
        print(f"💬 Messages: {data['total_messages']}")
        return data
    except FileNotFoundError:
        print(f"❌ File {filename} not found!")
        return None

# List saved conversations
import glob
saved_files = glob.glob("conversation_*.json")
if saved_files:
    print("📁 Saved conversations:")
    for file in saved_files:
        print(f"   • {file}")

In [ ]:
# Cell 14: Install Flask for web interface
!pip install flask flask-ngrok